In [2]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import classification_report
import pickle


In [11]:
df_EUROSTAT = pd.read_csv("EUROSTAT/data/serie/serie_EU.csv")
df_GTREND = pd.read_csv('GTrends/data/final/gtrend_monthly_2017.csv')
df_GDELT = pd.read_csv('GDELT/data/final/gdelt_monthly.csv')
    
data = pd.concat([df_GTREND, df_GDELT], axis=1)
data.head()
    

,Date,Media,Media_Education_busqueda_mensual,Media_Investments_busqueda_mensual,Media_Jobs_busqueda_mensual,Media_Mortage_busqueda_mensual,Media_Unemployement_benefits_busqueda_mensual,Media_Unemployments_busqueda_mensual,Media_Welfare_busqueda_mensual,Media_Investments_busqueda_mensual.1,...,Media_Volume_intensity_Financial_inclusion,Media_Volume_intensity_Health_Service,Media_Volume_intensity_Inclusive_Growth,Media_Volume_intensity_Social_inclusion,Media_average_tone_Education,Media_Average_tone_Enterprise_Development,Media_Average_tone_Financial_Inclusion,Media_Average_tone_Health_Service,Media_Average_tone_Inclusive_Growth,Media_Average_tone_Social_inclusion
0,2017-01-31,34.526316,34.526316,31.052632,70.631579,5.368421,7.263158,45.000000,0.000000,31.052632,...,0.000400,0.032663,0.001526,0.016526,-4.047779,3.672095,0.955005,-3.134963,0.792732,0.280137
1,2017-02-28,34.000000,34.000000,33.236842,68.526316,10.052632,11.263158,34.157895,0.000000,33.236842,...,0.000300,0.029216,0.000989,0.013747,-4.251689,4.221805,3.418921,-3.644742,3.066589,0.669737
2,2017-03-31,36.210526,36.210526,34.131579,66.421053,15.368421,8.631579,45.631579,4.333333,34.131579,...,0.000495,0.037516,0.001384,0.019537,-9.263563,5.326442,5.087337,-10.663121,7.076711,3.958305
3,2017-04-30,30.894737,30.894737,31.500000,60.157895,9.000000,6.052632,39.473684,0.000000,31.500000,...,0.000542,0.033011,0.001689,0.017421,-1.940011,3.883058,3.712237,-4.871816,2.563179,2.432684
4,2017-05-31,33.263158,33.263158,31.868421,61.578947,5.157895,3.105263,41.578947,0.000000,31.868421,...,0.000563,0.051711,0.007711,0.018816,-2.871253,4.308063,0.567563,-22.223179,1.172721,5.180226


In [14]:
data.drop(columns='Media',axis=1, inplace=True)

In [15]:
data.head()

,Date,Media_Education_busqueda_mensual,Media_Investments_busqueda_mensual,Media_Jobs_busqueda_mensual,Media_Mortage_busqueda_mensual,Media_Unemployement_benefits_busqueda_mensual,Media_Unemployments_busqueda_mensual,Media_Welfare_busqueda_mensual,Media_Investments_busqueda_mensual.1,Media_Jobs_busqueda_mensual.1,...,Media_Volume_intensity_Financial_inclusion,Media_Volume_intensity_Health_Service,Media_Volume_intensity_Inclusive_Growth,Media_Volume_intensity_Social_inclusion,Media_average_tone_Education,Media_Average_tone_Enterprise_Development,Media_Average_tone_Financial_Inclusion,Media_Average_tone_Health_Service,Media_Average_tone_Inclusive_Growth,Media_Average_tone_Social_inclusion
0,2017-01-31,34.526316,31.052632,70.631579,5.368421,7.263158,45.000000,0.000000,31.052632,70.631579,...,0.000400,0.032663,0.001526,0.016526,-4.047779,3.672095,0.955005,-3.134963,0.792732,0.280137
1,2017-02-28,34.000000,33.236842,68.526316,10.052632,11.263158,34.157895,0.000000,33.236842,68.526316,...,0.000300,0.029216,0.000989,0.013747,-4.251689,4.221805,3.418921,-3.644742,3.066589,0.669737
2,2017-03-31,36.210526,34.131579,66.421053,15.368421,8.631579,45.631579,4.333333,34.131579,66.421053,...,0.000495,0.037516,0.001384,0.019537,-9.263563,5.326442,5.087337,-10.663121,7.076711,3.958305
3,2017-04-30,30.894737,31.500000,60.157895,9.000000,6.052632,39.473684,0.000000,31.500000,60.157895,...,0.000542,0.033011,0.001689,0.017421,-1.940011,3.883058,3.712237,-4.871816,2.563179,2.432684
4,2017-05-31,33.263158,31.868421,61.578947,5.157895,3.105263,41.578947,0.000000,31.868421,61.578947,...,0.000563,0.051711,0.007711,0.018816,-2.871253,4.308063,0.567563,-22.223179,1.172721,5.180226


In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)  # No barajar para mantener la temporalidad

scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)

# Definir el número de pasos de tiempo (timesteps) y características (features)
n_timesteps = 3 # Define el número de pasos de tiempo
n_features = 24 # Define el número de características




In [16]:
df_EUROSTAT


,year_month,total
0,2017-03-01,212800.8
1,2017-06-01,215840.8
2,2017-09-01,219059.4
3,2017-12-01,220487.7
4,2018-03-01,221984.1
5,2018-06-01,226911.9
6,2018-09-01,227300.1
7,2018-12-01,228923.4
8,2019-03-01,229417.1
9,2019-06-01,233126.9


In [25]:
df_EUROSTAT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 2017-03-01 to 2023-12-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   total   28 non-null     float64
dtypes: float64(1)
memory usage: 448.0+ bytes


In [24]:
df_EUROSTAT.columns

Index(['total'], dtype='object')

In [27]:

df_EUROSTAT['year_month'] = pd.to_datetime(df_EUROSTAT.index)

# Establecer 'year_month' como índice
df_EUROSTAT.set_index('year_month', inplace=True)

# Reindexar a una frecuencia mensual
df_EUROSTAT = df_EUROSTAT['total'].resample('M').ffill()  # ffill para rellenar hacia adelante los valores de los trimestres

print(df_EUROSTAT)

year_month
2017-03-31    212800.8
2017-04-30    212800.8
2017-05-31    212800.8
2017-06-30    215840.8
2017-07-31    215840.8
                ...   
2023-08-31    267535.4
2023-09-30    268847.7
2023-10-31    268847.7
2023-11-30    268847.7
2023-12-31    271641.5
Freq: M, Name: total, Length: 82, dtype: float64


In [29]:
# Supongamos que tienes un DataFrame X con una serie temporal mensual
# Aquí creo un ejemplo ficticio para ilustrar el proceso
# Reemplaza este DataFrame con tus datos reales

# DataFrame X (serie temporal mensual)
X_data = {
    'time': pd.date_range(start='2020-01-01', periods=30, freq='M'),
    'value': range(30)
}
X = pd.DataFrame(X_data)
X.set_index('time', inplace=True)
X

,value
time,
2020-01-31,0
2020-02-29,1
2020-03-31,2
2020-04-30,3
2020-05-31,4
2020-06-30,5
2020-07-31,6
2020-08-31,7
2020-09-30,8


In [30]:
# Reorganizar X para tener 3 columnas (t-0, t-1, t-2)
n_rows = len(X) - 2  # número de filas después de la reorganización
X_transformed = pd.DataFrame(index=range(n_rows), columns=['t-0', 't-1', 't-2'])

for i in range(n_rows):
    X_transformed.loc[i] = [X.iloc[i, 0], X.iloc[i+1, 0], X.iloc[i+2, 0]]

print(X_transformed)

   t-0 t-1 t-2
0    0   1   2
1    1   2   3
2    2   3   4
3    3   4   5
4    4   5   6
5    5   6   7
6    6   7   8
7    7   8   9
8    8   9  10
9    9  10  11
10  10  11  12
11  11  12  13
12  12  13  14
13  13  14  15
14  14  15  16
15  15  16  17
16  16  17  18
17  17  18  19
18  18  19  20
19  19  20  21
20  20  21  22
21  21  22  23
22  22  23  24
23  23  24  25
24  24  25  26
25  25  26  27
26  26  27  28
27  27  28  29


In [31]:
X_data = {
    'time': pd.date_range(start='2020-01-01', periods=30, freq='M'),
    'value1': range(30),
    'value2': range(100, 130),
    # Agrega más columnas según sea necesario
}
X = pd.DataFrame(X_data)
X.set_index('time', inplace=True)

# Reorganizar X para tener 3 columnas por serie (t-0, t-1, t-2)
n_rows = len(X) - 2  # número de filas después de la reorganización
num_series = len(X.columns)  # número de series en X
columns = [f't-{i}' for i in range(num_series * 3)]  # nombres de las nuevas columnas

X_transformed = pd.DataFrame(index=range(n_rows), columns=columns)

for i in range(n_rows):
    for j in range(num_series):
        X_transformed.loc[i, [f't-{j*3}', f't-{j*3+1}', f't-{j*3+2}']] = [X.iloc[i, j], X.iloc[i+1, j], X.iloc[i+2, j]]

print(X_transformed)

   t-0 t-1 t-2  t-3  t-4  t-5
0    0   1   2  100  101  102
1    1   2   3  101  102  103
2    2   3   4  102  103  104
3    3   4   5  103  104  105
4    4   5   6  104  105  106
5    5   6   7  105  106  107
6    6   7   8  106  107  108
7    7   8   9  107  108  109
8    8   9  10  108  109  110
9    9  10  11  109  110  111
10  10  11  12  110  111  112
11  11  12  13  111  112  113
12  12  13  14  112  113  114
13  13  14  15  113  114  115
14  14  15  16  114  115  116
15  15  16  17  115  116  117
16  16  17  18  116  117  118
17  17  18  19  117  118  119
18  18  19  20  118  119  120
19  19  20  21  119  120  121
20  20  21  22  120  121  122
21  21  22  23  121  122  123
22  22  23  24  122  123  124
23  23  24  25  123  124  125
24  24  25  26  124  125  126
25  25  26  27  125  126  127
26  26  27  28  126  127  128
27  27  28  29  127  128  129
